<center>
    <img src="./media/title.png" alt="title" width="46%" height="46%">
</center>

<div style="display: flex; justify-content: space-between;">
     <div>
         <h2 style="color:lightseagreen;">Charles-Philippe Bernard</h2>
         <h3><a>@charlyouki</a><br>
         <h4>Senior Manager of Software Engineering</h4>
         <h4><ul><li>UX/UI Strategy</li><li>LLM integration and AI Strategy</li></ul></h4>
        <h2 style="color:lightgrey; font-family:'Times New Roman';">JP Morgan Chase</h2>
    </div>
    <div>
        <center>
        <h5><a>frangin2003.github.io</a><br>
            <img src="./media/qr-code.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
    <div>
        <center>
        <h5><a>frangin2003/google-devfest-glasgow-2023</a><br>
            <img src="./media/qr-code2.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
</div>

<br><img src="./media/blablabla.gif" width="25%" height="25%" loop="false"> <img src="./media/blablabla2v3.gif" width="70%" height="70%">
<br><br><br><br>

##### 🗝️ Key terms
 | 🥩 Raw models | GPT3.5, GPT4 |
 | --- | --- |
 | 🟦 Context window | Prompt+LLM output |
 | 📃 Instructions | Part of the prompt |
 | 💬 Prompt | Input of LLM call |
 | 🔫 Zero-shot | Zero examples in context |
 | 💽 Memory | Part of the prompt |
 | ⛓️ Chains | Combine multiple components |
 | 🧰 Tools | GraphQL tool, REST API tool, custom tools |
 | 🧠 Reasonning | Chain of thoughts, Tree of thoughts |
 | 🤖 AI agent | ReAct Zero Shot |
 | ↖️ Embeddings | OpenAI embeddings |


# <img src="./media/anatomyv3.gif" alt="Anatomy Image" style="width:80%; height:80%;">


<center>
<img src="./media/the_mona_lisa_case.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/thief.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/mona_lisa3.png" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/coptimus_title.png" width="50%" height="50%" loop="false">
</center>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/guard_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/curator_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/janitor_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/director_title.png" alt="Director" style="width:25%; height:25%;">
</div>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/dr_watts_on_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/inspector_camget_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/robby_sciuto_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/robodoc_title.png" alt="Director" style="width:25%; height:25%;">
</div>

Begin!

In [1]:
%%capture
from pydantic import BaseModel, Field
from typing import Tuple

from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import load_tools
from langchain.llms import OpenAI, Cohere, GPT4All, CTransformers
from langchain.chains import APIChain, RetrievalQA
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from IPython.display import display, update_display, HTML, Javascript, clear_output
from utils import pretty_print
from utils import convert_to_json
from utils import display_agent_response
from utils import get_agent_chain
from utils import create_human_message_for_gpt4vision
from utils import display_html_for_streaming
from utils import stream_chat_content
from utils import run_and_display_agent_response
from utils import get_retrieval_qa_pydantic_from_document
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

load_dotenv()

In [2]:
# 👮‍♂️ create the police station DB
import sqlite3
from datetime import datetime

# Connect to or create the SQLite database
conn = sqlite3.connect('police.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Cases')
cursor.execute('DROP TABLE IF EXISTS Suspects')
cursor.execute('DROP TABLE IF EXISTS Policemen')
cursor.execute('DROP TABLE IF EXISTS CasePolicemen')
cursor.execute('DROP TABLE IF EXISTS PolicemenGrade')
cursor.execute('DROP TABLE IF EXISTS PolicemenWages')


# Create tables for the database if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Cases (
        case_id INTEGER PRIMARY KEY AUTOINCREMENT,
        case_number TEXT,
        details TEXT,
        date_created DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Suspects (
        suspect_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        question TEXT,
        answer TEXT,
        date_created DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Policemen (
        policeman_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        rank TEXT,
        date_joined DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CasePolicemen (
        case_id INTEGER,
        policeman_id INTEGER,
        FOREIGN KEY (case_id) REFERENCES Cases (case_id),
        FOREIGN KEY (policeman_id) REFERENCES Policemen (policeman_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS PolicemenGrade (
        grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
        grade TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS PolicemenWages (
        wage_id INTEGER PRIMARY KEY AUTOINCREMENT,
        policeman_id INTEGER,
        year INTEGER,
        wage_amount REAL,
        FOREIGN KEY (policeman_id) REFERENCES Policemen (policeman_id)
    )
''')

# Insert data into the tables
grades_data = [("Inspector",), ("Sergeant",), ("Detective",)]
cursor.executemany('INSERT INTO PolicemenGrade (grade) VALUES (?)', grades_data)

policemen_data = [
    ("Inspector Charles", "Inspector", "2020-01-15"),
    ("Inspector Smith", "Sergeant", "2018-05-20"),
    ("Inspector Johnson", "Detective", "2017-11-10"),
]
cursor.executemany('INSERT INTO Policemen (name, rank, date_joined) VALUES (?, ?, ?)', policemen_data)

cases_data = [
    ("Case #54504005", "The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir. We currently assume the painting is stolen.", "2023-10-20"),
    ("Case #12345678", "A rare gemstone was stolen from a local jewelry store.", "2023-10-21"),
    ("Case #98765432", "A mysterious arson incident occurred at a historical library.", "2023-10-22"),
]

cursor.executemany('INSERT INTO Cases (case_number, details, date_created) VALUES (?, ?, ?)', cases_data)

suspects_data = [
    ("Rest Webber", "What time did you last check the Mona Lisa room?", "I did my usual rounds at 10 PM. Everything was in order.", "2023-10-20"),
    ("SQL Sally", "When did you last access the Mona Lisa room?", "I was last there at 7 PM for a general inspection.", "2023-10-20"),
    ("Victor Vectors", "When did you last clean the Mona Lisa room?", "I haven't cleaned there for a week.", "2023-10-20"),
    ("GraphQL Graeme", "Did you have any issues with the Museum systems recently?", "We had some issues today, I was not able to enter my office, but Victor, who used to work in IT, has sorted it out. Bless him!", "2023-10-20"),
    ("Lucy Larceny", "What were you doing near the jewelry store?", "I was just passing by, I swear!", "2023-10-21"),
    ("Arson Al", "Did you see anyone suspicious at the library?", "I saw someone running away, but couldn't identify them.", "2023-10-22"),
]

cursor.executemany('INSERT INTO Suspects (name, question, answer, date_created) VALUES (?, ?, ?, ?)', suspects_data)

case_policemen_data = [
    (1, 1),  # Inspector Charles on Mona Lisa case
    (2, 2),  # Inspector Smith on Gemstone case
    (3, 3),  # Inspector Johnson on Library arson case
    (2, 1),  # Inspector Charles also on Gemstone case
    (3, 2),  # Inspector Smith also on Library arson case
]

cursor.executemany('INSERT INTO CasePolicemen (case_id, policeman_id) VALUES (?, ?)', case_policemen_data)

# Insert dummy data for policeman wages
wages_data = [
    (1, 2021, 55000.0),
    (1, 2022, 56000.0),
    (1, 2023, 57000.0),
    (2, 2021, 50000.0),
    (2, 2022, 51000.0),
    (2, 2023, 52000.0),
    (3, 2021, 60000.0),
    (3, 2022, 61000.0),
    (3, 2023, 62000.0),
]
cursor.executemany('INSERT INTO PolicemenWages (policeman_id, year, wage_amount) VALUES (?, ?, ?)', wages_data)


# Commit the changes and close the connection
conn.commit()
conn.close()


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE POLICE STATION REST API ⚠️</h1>
</div>


In [3]:
# 🏛️ create the museum room activity DB
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('museum_activity.db')
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS Employees")
cursor.execute("DROP TABLE IF EXISTS RoomAccess")

# Create tables
cursor.execute("""
CREATE TABLE Employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    role TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE RoomAccess (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    room TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    comment TEXT NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(id)
);
""")

# Insert data
cursor.executemany("INSERT INTO Employees (name, role) VALUES (?, ?)", [
    ('Rest Webber', 'Night Guard'),
    ('SQL Sally', 'Curator'),
    ('Victor Vectors', 'Janitor'),
    ('GraphQL Graeme', 'Museum Director')
])

cursor.executemany("INSERT INTO RoomAccess (employee_id, room, timestamp, comment) VALUES (?, ?, ?, ?)", [
    (4, 'Museum Office', '2023-10-01 07:00', 'Error: Access Not Working'),
    (1, 'Mona Lisa Room', '2023-10-01 22:00', 'Accessed'),
    (2, 'Mona Lisa Room', '2023-10-01 19:00', 'Accessed'),
])

# Commit and close
conn.commit()
conn.close()


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE MUSEUM ROOM ACTIVITY GRAPHQL API ⚠️</h1>
</div>


<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;border: none;'><img style="transform: scaleX(-1);border-radius: 15px;" src="./media/dr_watts_on_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color:orange;">Dr. Watts-On - archive/documentation/secretary AI agent</h2>
<h4>Interacts with the  Police REST API to retrieve and inserts information in the Police DB like:
<ul><li>other agents reports</li><li>case details</li><li>suspect list and interviews...</li></ul>Falls back to doing SQL queries directly to the DB when the REST API is down</td></h4>
</tr></table>

In [2]:
llm4watson = ChatOpenAI(model_name="gpt-3.5-turbo-16k", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# 📃 custom instructions
prefix4watson = """You are Dr. Watts-On, archive/documentation/secretary AI agent working for the police force.
Your mission is to interact with the Police REST API to retrieve and inserts information in the Police DB like:
- other agents reports
- case details
- suspect list and interviews...
When you respond, make sure to add break lines after each sentence.
If the user is engaging a casual discussion, do not use tools, just respond back directly.
If the REST API is down, you fall back to doing SQL queries directly to the DB when the REST API is down."""

In [4]:
# 🧰 define the tools

# 🕵️ Police Report API APIChain Tool
POLICE_REPORTING_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

Method: GET
Path: /policemen
Description: Retrieves a list of all policemen from the database.

Method: GET
Path: /policemen/<policeman_id>
Description: Fetches details of a specific policeman by their ID.

Method: GET
Path: /cases/<case_id>
Description: Retrieves details of a specific case, including suspects, the assigned policeman, and the policeman's grade.

Method: POST
Path: /cases
Description: Allows the creation of a new case by providing case details.
Python Example:
import requests
new_case_data = {
'case_number': 'Case #123456',
'details': 'Details of the new case',
'date_created': '2023-11-22'
}
response = requests.post("http://localhost:5000/cases", json=new_case_data)

Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message."""

policeAPIChain = APIChain.from_llm_and_api_docs(llm4watson, POLICE_REPORTING_API_DOCS, limit_to_domains=[
    "http://localhost:5000/",
    ], verbose=True)

# 🏛️ Museum DB SQLChain Tool
police_db = SQLDatabase.from_uri("sqlite:///police.db")
police_db_chain = SQLDatabaseChain.from_llm(llm4watson, police_db, verbose=True)

def policeDB(input: str) -> str:
    return police_db_chain.run(input)

def policeAPI(input: str) -> str:
    return policeAPIChain.run(input)

tools4watson = [
    Tool(
        name='Police REST API',
        func= lambda string: policeAPI(string),
        description=
"""Utilize the REST API to manage the police DB."""
    ),
    Tool(
        name='Police DB',
        func= lambda string: policeDB(string),
        description=
"""Access the Police SQL database."""
    )
]

In [5]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4watson = get_agent_chain(llm4watson, prefix4watson, tools4watson)

In [6]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Hello, who are you?", True)




> Entering new AgentExecutor chain...
Thought: This is a casual discussion, so I can respond directly without using any tools.
Action: N/A
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:I need to provide a response directly without using any tools.
Final Answer: I am Dr. Watts-On, an AI agent working for the police force.

> Finished chain.


In [80]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Get me the case id=1 and print it in a nice readable way")



> Entering new AgentExecutor chain...
Thought: The user wants to retrieve the details of a specific case and display it in a readable format.
Action: Police REST API
Action Input: GET /cases/1

> Entering new APIChain chain...
http://localhost:5000/cases/1
{
  "case": {
    "case_id": 1,
    "case_number": "Case #54504005",
    "date_created": "2023-10-20",
    "details": "The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir. We currently assume the painting is stolen."
  },
  "policeman": {
    "date_joined": "2020-01-15",
    "grade": "Inspector",
    "name": "Inspector Charles",
    "policeman_id": 1,
    "rank": "Inspector"
  },
  "suspects": [
    {
      "answer": "I did my usual rounds at 10 PM. Everything was in order.",
      "date_created": "2023-10-20",
      "name": "Rest Webber",
      "question": "What time did you last check the Mona Lisa room?",
      "suspect_id": 1
    },
    {
      "answer": "I was last there at 7 PM 

,"The details of case ID 1 are as follows: - Case ID: 1 - Case number: ""Case #54504005"" - Date created: ""2023-10-20"" - Details: ""The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir. We currently assume the painting is stolen."" - Policeman details: - Policeman ID: 1 - Name: ""Inspector Charles"" - Rank: ""Inspector"" - Date joined: ""2020-01-15"" - Grade: ""Inspector"" - Suspects details: - Suspect 1: - Suspect ID: 1 - Name: ""Rest Webber"" - Question: ""What time did you last check the Mona Lisa room?"" - Answer: ""I did my usual rounds at 10 PM. Everything was in order."" - Date created: ""2023-10-20"" - Suspect 2: - Suspect ID: 2 - Name: ""SQL Sally"" - Question: ""When did you last access the Mona Lisa room?"" - Answer: ""I was last there at 7 PM for a general inspection."" - Date created: ""2023-10-20"" - Suspect 3: - Suspect ID: 3 - Name: ""Victor Vectors"" - Question: ""When did you last clean the Mona Lisa room?"" - Answer: ""I haven't cleaned there for a week."" - Date created: ""2023-10-20"" - Suspect 4: - Suspect ID: 4 - Name: ""GraphQL Graeme"" - Question: ""Did you have any issues with the Museum systems recently?"" - Answer: ""We had some issues today, I was not able to enter my office, but Victor, who used to work in IT, has sorted it out. Bless him!"" - Date created: ""2023-10-20"""


<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robby_sciuto_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Robby Sciuto - IT System Expert</h2>
<h4>Hacks through IT systems related to a crime case to report any suspicious activities that could be linked to the case</h4></td>
</tr></table>

In [ ]:
# will do 2 graphql queries and then will try to interact with the SQl guy and there is an outtage

<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robodoc_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Robodoc - Document Analysis Expert</h2>
<h4>Uses RAG to perform thorough analysis through pile of documents to report anything unusual from suspects information</h4></td>
</tr></table>

In [2]:
llm4robodoc = ChatOpenAI(model_name="gpt-3.5-turbo-16k", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# 📃 custom instructions
system_instructions4robodoc = """You are Robodoc, a police bot expert in document analysis.
You are here to help cracking a crime case.
You will be given a suspect's name and his latest 3 months bank statement.
Your role is to analyze the statement and find out anything suspicious and report back using a specific format.
"""

In [4]:
# 🗂️ Response Schema
class ReportSchema(BaseModel):
    """An answer to the question being asked, with sources."""
    suspectName: str = Field(..., description="Name of the suspect") 
    report_content: str = Field(..., description="Analize the document, look for anything suspicious, describe with a sentence the most suspicious here. If not found say 'N/A'")
    suspiscious: bool = Field(..., description="True if the suspect has anything suspiscious found")

In [5]:
retrieval_qa_pydantic = get_retrieval_qa_pydantic_from_document(llm4robodoc, system_instructions4robodoc, 'victor_bank_statement.csv', ReportSchema)
result = retrieval_qa_pydantic.run("Give your report about Victor Vectors")
output = json.dumps(result.dict(), indent=4)
display_agent_response('robodoc', output)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [8]:
documents = ['bank_statements/graeme_bank_statement.csv',
             'bank_statements/sally_bank_statement.csv',
             'bank_statements/victor_bank_statement.csv',
             'bank_statements/webber_bank_statement.csv']
names = ['Grafcuel Graeme', 'Sikwel Sally', 'Victor Vectors', 'Rest Webber']

for doc, name in zip(documents, names):
    retrieval_qa_pydantic = get_retrieval_qa_pydantic_from_document(llm4robodoc, system_instructions4robodoc, doc, ReportSchema)
    result = retrieval_qa_pydantic.run(f"Give your report about {name}")
    output = json.dumps(result.dict(), indent=4)
    display_agent_response('robodoc', output)


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/inspector_camget_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Inspector Camget -  Crime scene analysis agent</h2>
<h4>
<ul><li>Uses GPT4-Vision to report anything unusual on the crime scene</li><li>Cross-checks suspect pictures one by one with his analysis</li><li>Creates a report and sends it to Dr. Watts-On</li></ul></h4></td>
</tr></table>

In [9]:
chat4camget = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)

In [ ]:
# 📃 custom instructions
instructions4camget_crime_scene_analysis = """
**Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

You are Inspector Camget, an AI police robot that specializes in crime scene analysis.
The current case you are analyzing is about the theft of the painting "The Mona Lisa" at Le Louvre museum.
Please report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

Report format is:
** Report start **
1. Sentence of observation #1
2. Sentence of observation #2
...
** Report end **

Take a deep breath and give your report"""


In [8]:
image_crime_scene = 'http://frangin2003.free.fr/devfest/mona_lisa3.png'
msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_crime_scene_analysis, image_crime_scene)
output_camget001 = chat4camget.invoke([msg_camget001])
display_agent_response('inspector_camget', output_camget001.content)


In [12]:
# put the above report in a file
crime_scene_report = """** Report start **
1. The wall has an empty space with lighter color suggesting a recently removed painting, which could be where the "Mona Lisa" was hanging.
2. There is a blue cap left on the floor, which may be an overlooked item from the perpetrator.
3. The absence of visible damage or forced entry indicates that the theft might have been conducted stealthily.
** Report end **"""

pictures = [
    'http://frangin2003.free.fr/devfest/director.png',
    'http://frangin2003.free.fr/devfest/curator.png',
    'http://frangin2003.free.fr/devfest/janitor.png',
    'http://frangin2003.free.fr/devfest/guard.png'
]

for picture, name in zip(pictures, names):

    instructions4camget_suspect_analysis = f"""
    **Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

    You are Inspector Camget, an AI police robot that specializes in suspect analysis.
    You will be given:
    - a text report 'crime scene analysis' describing anything suspiscious you found at the crime scene
    - the suspect name
    - the picture of a suspect

    Cross check 'crime scene analysis' report with the suspect picture and report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
    Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

    Report format is:
    ** Report start **
    Name: The name of the suspect
    Status: Suspicious / Not suspicious
    1. Sentence of observation #1
    2. Sentence of observation #2
    ...
    ** Report end **

    The suspect name is {name}

    The 'crime scene analysis' is:
    {crime_scene_report}

    Take a deep breath and give your report"""

    msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_suspect_analysis, picture)
    output_camget001 = chat4camget.invoke([msg_camget001])
    display_agent_response('inspector_camget', output_camget001.content)



 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🐣 CRACKING THE CASE 🐣</h1>
</div>


In [ ]:
llm4optimus = ChatOpenAI(model_name="gpt-4-1106-preview", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# 📃 custom instructions
prefix4optimus = """You are Dr. Watts-On, archive/documentation/secretary AI agent working for the police force.
Your mission is to interact with the Police REST API to retrieve and inserts information in the Police DB like:
- other agents reports
- case details
- suspect list and interviews...
When you respond, make sure to add break lines after each sentence.
If the user is engaging a casual discussion, do not use tools, just respond back directly.
If the REST API is down, you fall back to doing SQL queries directly to the DB when the REST API is down."""

In [ ]:
# 🧰 define the tools

policeAPIChain = APIChain.from_llm_and_api_docs(llm4watson, POLICE_REPORTING_API_DOCS, limit_to_domains=[
    "http://localhost:5000/",
    ], verbose=True)

# 🏛️ Museum DB SQLChain Tool
police_db = SQLDatabase.from_uri("sqlite:///police.db")
police_db_chain = SQLDatabaseChain.from_llm(llm4watson, police_db, verbose=True)

def policeDB(input: str) -> str:
    return police_db_chain.run(input)

def policeAPI(input: str) -> str:
    return policeAPIChain.run(input)

tools4coptimus = [
    Tool(
        name='Police REST API',
        func= lambda string: policeAPI(string),
        description=
"""Utilize the REST API to manage the police DB."""
    ),
    Tool(
        name='Police DB',
        func= lambda string: policeDB(string),
        description=
"""Access the Police SQL database."""
    )
]

In [ ]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4coptimus = get_agent_chain(llm4coptimus, prefix4coptimus, tools4coptimus)

In [ ]:
run_and_display_agent_response('coptimus', agent_chain4coptimus, "Crack the case", True)


<br><br><br><br><br><br><br><br><br>

<div style="display: flex; justify-content: space-between;">
    <div style="width: 40%;">
        <img src="./media/simulacra.png" alt="Simulacra" style="width:100%; height:auto;">
    </div>
    <div style="width: 60%;">
         <h1>📃 Some Papers</h1>
         <ul>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2304.03442">Generative Agents: Interactive Simulacra of Human Behavior</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.16291">Voyager: An Open-Ended Embodied Agent with Large Language Models</a></li>
             <li><a style="color:yellow" href="https://fablestudio.github.io/showrunner-agents/">SHOW-1 and Showrunner Agents in Multi-Agent Simulations</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.03427">TPTU: Task Planning and Tool Usage</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2309.10691">MINT: Evaluating LLMs in Multi-turn Interaction with Tools and Language Feedback</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2303.09014">ART: Automatic multi-step reasoning and tool-use</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.01990">From Prompt Injections to SQL Injection Attacks</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.11853">How to Prompt LLMs for Text-to-SQL</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.03111">Can LLM Already Serve as A Database Interface</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2310.04474">Reverse Chain: A Generic-Rule for LLMs to Master Multi-API Planning</a></li>
         </ul>
    </div>
</div>


<div style="display: flex; justify-content: space-between;">
    <div style="width: 50%;">
        <img src="./media/minecraft.png" alt="Minecraft" style="width:80%; height:auto;">
    </div>
    <div style="width: 50%;">
        <img src="./media/southpark.png" alt="Southpark" style="width:80%; height:auto;">
    </div>
</div>


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🤗 THANKS FOR YOUR ATTENTION 🤗</h1>
</div>

<center>
<img src="./media/thankyou.png" width="40%" height="40%" loop="false">
</center>

<br><br><br><br><br><br><br>

<center>
<img src="./media/questions.png" width="40%" height="40%" loop="false">
</center>